In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import csv
import math
from pandas.api.types import CategoricalDtype
import glob
from collections import defaultdict
pd.set_option('display.max_columns', None)
params=['submap keyframes','k strongest','z min','resolution r [m]','Weight','loss limit','dataset','method']
error_metrics=["Transl. Error[%]", "Rot.err.[deg/100m]", "RPE(m)", "ATE(m)"]
datasets=["oxford","mulran"]
sns.set_theme(style="ticks", color_codes=True)
def LoadData(prefix,base_dir):
    search_string="/"+prefix+"*.csv"
    dfs = []
    for filename in glob.glob(base_dir+search_string, recursive=True):
        print(filename)
        df = pd.read_csv(filename, index_col=None, header=0, skipinitialspace=True)
        dfs.append(df)
        df = pd.concat(dfs, axis=0, ignore_index=True)
        df = df.rename(columns={'resolution r': 'resolution r [m]'})
    df = df.rename(columns={'Trans.err.(%)': 'Transl. Error[%]'})
    df = df.rename(columns={'Rot.err.(deg/100m)': 'Rot.err.[deg/100m]'})

    df['for_copy'] = df.apply (lambda row: "{:2.2f}".format(row["Transl. Error[%]"])+"/"+"{:2.2f}".format(row["Rot.err.[deg/100m]"]) , axis=1)
    df['for_copy_ATE'] = df.apply (lambda row: "{:2.2f}".format(row["RPE(m)"])+"/"+"{:2.2f}".format(row["ATE(m)"]) , axis=1)
    df['method'].replace(['baseline-cfear-1'], 'grid-search-cfear-1') #Correct wrong namer
    df['method'].replace(
        to_replace=['baseline-cfear-1'],
        value='grid-search-cfear-1',
        inplace=True
    )
    print("WARNING WARNING, THIS PART DOES NOT WORK IN GENERAL, USE WITH CARE FOR NEW DATASETS")
    df = df.drop(df[df["Transl. Error[%]"]<1].index)
    
    
    df=df.sort_values(by=params)

    return df
    
###### LOAD DATA ############
#baseline='/home/daniel/rosbag/CFEAR_EVAL/eval_storage/8_grid_search_2022-04-04_2143'
baseline='./data/'

df1=LoadData("grid-search-cfear-1", baseline)
df2=LoadData("grid-search-cfear-2", baseline)
df3=LoadData("grid-search-cfear-3", baseline)
dfs=[df1,df2,df3]
#for df in dfs:
    #print(df)


    #params=['submap keyframes','k strongest','z min','resolution r [m]','Weight','loss limit']
    #full_params=params+['dataset','method']
    #df=df.sort_values(by=full_params)


./data/grid-search-cfear-1_2022-04-10_1134_eval.csv
./data/grid-search-cfear-1_2022-04-10_1136_eval.csv
WARNING WARNING, THIS PART DOES NOT WORK IN GENERAL, USE WITH CARE FOR NEW DATASETS
./data/grid-search-cfear-2_2022-04-10_1803_eval.csv
./data/grid-search-cfear-2_2022-04-10_1422_eval.csv
WARNING WARNING, THIS PART DOES NOT WORK IN GENERAL, USE WITH CARE FOR NEW DATASETS
./data/grid-search-cfear-3_2022-04-12_1057_eval.csv
./data/grid-search-cfear-3_2022-04-14_1154_eval.csv
WARNING WARNING, THIS PART DOES NOT WORK IN GENERAL, USE WITH CARE FOR NEW DATASETS


In [2]:
#################### NOTE, THIS SCRIPT TAKES TIME TO RUN - a few minutes ##############
for i in range(len(dfs)):
    
    methods=np.array(dfs[i]["method"].unique())
    datasets=np.array(dfs[i]["dataset"].unique())
    print("------------SCV------------")

    imin=dfs[i]['Transl. Error[%]'].argmin()
    print(datasets)
    print(methods)
    error=dfs[i].iloc[imin]["Transl. Error[%]"]
    print(error)
    #if(error<1)
    
    
    
    

------------SCV------------
['mulran' 'oxford']
['grid-search-cfear-1']
1.432
------------SCV------------
['mulran' 'oxford']
['grid-search-cfear-2']
1.227
------------SCV------------
['mulran' 'oxford']
['grid-search-cfear-3']
1.131


In [3]:
# Functions for reducing dataframe size by averaging the error over all trajectories.
count = 0
columns=params+error_metrics
table=[]
def getParVek(row):
    curr_pars=[]
    for par in params:
        curr_pars.append(row[par])
    return curr_pars


def CreateMinimalDataframe(dtmp):
    dconcat=pd.DataFrame( columns=params+error_metrics)
    
    prev_pars=[]
    count = 0
    idx_vek=[]    
    for current_index, row in dtmp.iterrows():
        curr_pars=getParVek(row)
        #print(curr_pars)
        
        if(count==0 or prev_pars==curr_pars):
            idx_vek.append(current_index) 
        else:
            df_curr=dtmp.loc[idx_vek]
            #print(df_curr["dataset"])
            errors=[]
            for error in error_metrics:
                err=df_curr[error].mean()
                errors.append(err)
            idx_vek.clear()
            idx_vek.append(current_index)            
            dd = pd.DataFrame([[int(count)]+prev_pars+errors], columns = ["id"]+columns)
            dconcat=dconcat.append(dd)
            #print(dd)
            
        
            
        prev_pars=curr_pars
        count = count+1
    df_curr=dtmp.loc[idx_vek]
    #print(idx_vek)
    errors=[]
    for error in error_metrics:
        err=df_curr[error].mean()
        errors.append(err)
    idx_vek.clear()
    dd = pd.DataFrame([[int(count)]+prev_pars+errors], columns = ["id"]+columns)
    dconcat=dconcat.append(dd)
    dconcat=dconcat.set_index("id")
    dconcat.reset_index() 
         
        
    return dconcat


In [4]:
######### Reduces frames to minimal representation
dfs_filtered=[]

for di in dfs:
    dfiltered=CreateMinimalDataframe(di)
    dfs_filtered.append(dfiltered)
    print(str(dfiltered.shape[0]))



576
576
972


In [5]:
######### Prepare string output
error_metrics=["Transl. Error[%]", "Rot.err.[deg/100m]", "RPE(m)", "ATE(m)"]
for i in range(len(dfs_filtered)):
    dfs_filtered[i]["mean"] = dfs_filtered[i].apply (lambda row: "{:2.2f}".format(row["Transl. Error[%]"])+"/"+"{:2.2f}".format(row["Rot.err.[deg/100m]"]) , axis=1)         
    dfs_filtered[i]["Mean RPE [cm]/Mean ATE [m]"] = dfs_filtered[i].apply (lambda row: "{:2.2f}".format(100*row["RPE(m)"])+"/"+"{:2.2f}".format(row["ATE(m)"]) , axis=1)         
    dfs_filtered[i]["mean SCV"] = dfs_filtered[i].apply (lambda row: "N/A" , axis=1)         
    dfs_filtered[i]["mean Opti."] = dfs_filtered[i].apply (lambda row: "N/A" , axis=1)         


In [6]:
dfs_filtered_copy = dfs_filtered.copy()

for i in range(len(dfs_filtered_copy)):
    
    df=dfs_filtered_copy[i]
    #print(df)
    for dataset_eval in datasets:
        print("-----------------------------------------")
        print("Evaluation dataset: " + dataset_eval)
        #for method in methods:
        for dataset_training in datasets:
            if dataset_training != dataset_eval:
                # GET METHOD NAME AND SUBDIVIDE PER DATASET
                method=df["method"].unique()[0]
                df_training=df[(df["dataset"]==dataset_training)] # Subset
                df_eval=df[ (df["dataset"]==dataset_eval)]

                #Get training error
                idx_argmin=df_training['Transl. Error[%]'].argmin()
                training_argmin_frame=df_training.iloc[idx_argmin]
                training_error_copy = training_argmin_frame["mean"]
                training_error = str(training_argmin_frame["Transl. Error[%]"]) 
                   
                #Get the corresponding frame but for the other dataset ( df_eval["dataset"]!=training_argmin_frame["dataset"] )
                mask=(df_eval["submap keyframes"]==training_argmin_frame["submap keyframes"]) & (df_eval["k strongest"]==training_argmin_frame["k strongest"]) & (df_eval["z min"]==training_argmin_frame["z min"]) & (df_eval["resolution r [m]"]==training_argmin_frame["resolution r [m]"]) & (df_eval["Weight"]==training_argmin_frame["Weight"]) & (df_eval["method"]==training_argmin_frame["method"]) & (df_eval["loss limit"]==training_argmin_frame["loss limit"]) & (df_eval["dataset"]!=training_argmin_frame["dataset"])
                eval_argmin_frame=df_eval[mask].iloc[0]
                evaluation_error_copy = eval_argmin_frame["mean"]
                evaluation_error_drift = eval_argmin_frame["Transl. Error[%]"]
                evaluation_error_rpe=eval_argmin_frame["Mean RPE [cm]/Mean ATE [m]"]
                #df_training=df_training.set_index(range(0,df_training.shape[0]))
                #print(df_training)
                eval_idx_list=[]
                train_idx_list=[]
                for current_index, row in df_eval.iterrows():
                    eval_idx_list.append(current_index)
                df.loc[eval_idx_list,"mean SCV"] = evaluation_error_copy
                
                for current_index, row in df_training.iterrows():
                    train_idx_list.append(current_index)
                df.loc[train_idx_list,"mean Opti."] = training_error_copy
                                
                #Optional prints
                print("Training error: " + training_error_copy)
                #print(training_argmin_frame)
                print("Evaluation error: " + evaluation_error_copy)
                #print(eval_argmin_frame)

                #print("method: "+method+", "+dataset_training+", Mean Opti. ="+training_error_copy)
                #print("method: "+method+", ,"+dataset_eval+", Mean SCV. ="+evaluation_error_copy)
                #print("method: "+method+", ,"+dataset_eval+", RPE/ATE SCV. ="+evaluation_error_rpe)
                df.to_csv('./output/'+method+'.csv', index=False)  
        

for dataset_eval in datasets:
    print("Evaluated dataset: " + dataset_eval)
    for df in dfs_filtered_copy:
        drow=df[df["dataset"]==dataset_eval].iloc[0]
        #print(drow)
        print( "Method: "+drow["method"]+", mean SCV and Opti. & " + drow["mean SCV"] + " & " + drow["mean Opti."])
        
    
        
    

-----------------------------------------
Evaluation dataset: mulran
Training error: 1.63/0.57
Evaluation error: 2.34/0.79
-----------------------------------------
Evaluation dataset: oxford
Training error: 2.34/0.79
Evaluation error: 1.69/0.60
-----------------------------------------
Evaluation dataset: mulran
Training error: 1.40/0.49
Evaluation error: 1.93/0.66
-----------------------------------------
Evaluation dataset: oxford
Training error: 1.93/0.66
Evaluation error: 1.40/0.49
-----------------------------------------
Evaluation dataset: mulran
Training error: 1.20/0.39
Evaluation error: 1.60/0.57
-----------------------------------------
Evaluation dataset: oxford
Training error: 1.55/0.56
Evaluation error: 1.22/0.39
Evaluated dataset: mulran
Method: grid-search-cfear-1, mean SCV and Opti. & 2.34/0.79 & 2.34/0.79
Method: grid-search-cfear-2, mean SCV and Opti. & 1.93/0.66 & 1.93/0.66
Method: grid-search-cfear-3, mean SCV and Opti. & 1.60/0.57 & 1.55/0.56
Evaluated dataset: o